In [2]:
%load_ext rpy2.ipython
%load_ext watermark
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

In [4]:
from pathlib import Path
import os
import numpy as np
from numpy import random
import statsmodels.api as sm
import warnings
import pandas as pd
import scipy.stats as ss


%watermark -a 'Bo Sun' -d -t -v -p numpy,pandas,statsmodels,scipy

Bo Sun 2019-06-11 18:57:15 

CPython 3.7.3
IPython 7.4.0

numpy 1.15.4
pandas 0.24.2
statsmodels 0.9.0
scipy 1.1.0


# This notebook demos the workflow of performing eQTLs with python. 
## The inputs are supposed to be in a matrix format that is commonly used in RNA-seq analysis (represented as pandas.DataFrame).

In [3]:
expr_file = '/Users/bos/repos/QR-eQTL/data/example/expr_TPM.csv'

In [4]:
covar = pd.read_csv('/Users/bos/repos/QR-eQTL/data/example/covariates.csv')

In [5]:
var = pd.read_csv('/Users/bos/repos/QR-eQTL/data/example/variant_genotype.csv')

## Gene expression Matrix: rows are genes, columns are samples

In [6]:
expr = pd.read_csv(expr_file, index_col=0)

In [7]:
expr.head()

,SAMPLE-111FC,SAMPLE-111VG,SAMPLE-1122O,SAMPLE-1128S,SAMPLE-113JC,SAMPLE-117XS,SAMPLE-117YW,SAMPLE-117YX,SAMPLE-1192W,SAMPLE-1192X,...,SAMPLE-ZXG5,SAMPLE-ZY6K,SAMPLE-ZYFC,SAMPLE-ZYFD,SAMPLE-ZYFG,SAMPLE-ZYT6,SAMPLE-ZYW4,SAMPLE-ZYY3,SAMPLE-ZZ64,SAMPLE-ZZPT
gene_id,,,,,,,,,,,,,,,,,,,,,
ENSG00000142621.15,0.093217,0.190204,0.045843,0.067006,0.146542,0.079840,0.161716,0.132957,0.132377,0.109331,...,0.074321,0.041693,0.144628,0.387563,0.007511,0.055692,0.279078,0.510283,0.098275,0.154692
ENSG00000121905.5,0.052895,0.172735,0.312157,0.202859,0.462697,0.000000,0.234964,0.190650,0.240397,0.330911,...,0.196843,0.504802,0.268626,0.324711,0.238720,0.072256,0.095014,0.126907,0.250950,0.324160
ENSG00000269501.1,0.180958,0.159978,0.044493,0.108359,0.123575,0.154912,0.062784,0.516224,0.102778,0.129647,...,0.084138,0.094414,0.102056,0.127264,0.102051,0.092651,0.324877,0.149266,0.154907,0.150074
ENSG00000150337.9,0.000000,0.644279,0.240960,0.215207,0.223131,0.116551,0.090680,0.147118,0.231987,0.170189,...,0.043397,0.462545,0.115173,0.501281,0.052631,0.055757,0.366613,0.000000,0.043043,0.083372
ENSG00000186844.4,361.304053,507.286405,5409.122095,369.807914,320.052938,678.431577,172.180267,707.136790,142.206534,324.450775,...,468.881135,600.987049,433.663381,903.572590,1137.315847,2328.096303,242.284989,536.726209,591.425214,548.232037


## Covariates: confounding variables, could be numeric or categorical

In [8]:
covar = pd.read_csv('/Users/bos/repos/QR-eQTL/data/example/covariates.csv')

In [9]:
covar.tail()

,ID,SAMPLE-111FC,SAMPLE-111VG,SAMPLE-1122O,SAMPLE-1128S,SAMPLE-113JC,SAMPLE-117XS,SAMPLE-117YW,SAMPLE-117YX,SAMPLE-1192W,...,SAMPLE-ZXG5,SAMPLE-ZY6K,SAMPLE-ZYFC,SAMPLE-ZYFD,SAMPLE-ZYFG,SAMPLE-ZYT6,SAMPLE-ZYW4,SAMPLE-ZYY3,SAMPLE-ZZ64,SAMPLE-ZZPT
60,C60,-0.124125,-0.041981,-0.098107,-0.009964,-0.071172,-0.077161,0.037342,-0.051677,-0.042433,...,-0.073631,-0.154109,-0.009641,-0.138827,-0.251305,-0.066708,0.087597,0.073375,0.075288,-0.186513
61,C61,-0.020212,-0.048675,-0.131256,0.101649,0.011187,0.001656,-0.192751,0.062457,-0.186217,...,-0.046109,-0.079703,-0.070435,0.135139,0.093878,0.153023,-0.164157,-0.065934,-0.078009,0.239288
62,C62,-0.006455,0.064459,-0.085590,-0.064206,-0.049075,-0.043611,0.179756,0.036066,0.005312,...,-0.150670,0.235637,-0.048191,0.025079,0.155109,-0.098938,-0.069294,-0.057353,0.002181,-0.059692
63,sex,1.000000,1.000000,2.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,2.000000,1.000000,1.000000
64,platform,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Variants nearby each gene: genotype(GT) is the allele dosage for a given variant.

In [10]:
var = pd.read_csv('/Users/bos/repos/QR-eQTL/data/example/variant_genotype.csv')

In [11]:
var.head()

,gene_id,variant_id,GT-1,GT-2,GT-3,GT-4,GT-5,GT-6,GT-7,GT-8,...,GT-405,GT-406,GT-407,GT-408,GT-409,GT-410,GT-411,GT-412,GT-413,GT-414
0,ENSG00000065325.8,X_1287943_C_G,0,1,0,0,0,0,0,0,...,0,0,0,0,2,0,0,1,0,0
1,ENSG00000065325.8,19_7674782_C_G,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,2
2,ENSG00000065325.8,11_8682249_C_G,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ENSG00000069011.11,20_12222898_C_G,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ENSG00000069011.11,14_12485031_C_G,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## For each Gene-Variant pair, Perform the Linear regression with specified model.
 - OLS
 - QuantReg: quantile regression
 
## Command Line usage:
`3`

In [12]:
lm = sm.QuantReg  # or sm.OLS

In [13]:
def write_output(fit, fout):
    line = "\t".join([row['gene_id'], row['variant_id'], str(fit.pvalues[-1]), str(fit.params[-1]), str(fit.tvalues[-1])])
    fout.write(line + '\n')

In [14]:
output = open('/Users/bos/repos/QR-eQTL/data/example/sample_output.txt', 'w')
output.write('gene_id\t' + 'variant_id\t' + 'pvals\t' + 'slope\t' + 'tvals\n')
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")  # Hide annoying warning: IterationLimitWarning: Maximum number of iterations (50) reached.
    for idx, row in var.iterrows():
        Y = expr.loc[row['gene_id']].values
        X = np.c_[covar.T.values[1:, :], row.filter(like='GT')]  # covar.T.values[1:, :] => removed the header
        X = sm.add_constant(X).astype('float')  # add intercept

        mod = lm(Y, X)
        if lm is sm.QuantReg:
            fit = mod.fit(max_iter=50)
        else:
            fit = mod.fit()
        if (idx+1) % 5 == 0:
            print(f'Finished for {idx+1} pairs.')
        write_output(fit, output)

    
output.close()

Finished for 5 pairs.
Finished for 10 pairs.
Finished for 15 pairs.
Finished for 20 pairs.
Finished for 25 pairs.


## Check output format: columns are gene, variant, p-value, slope(coefficient estimated for genotype), and t-value.

In [15]:
x = pd.read_csv('/Users/bos/repos/QR-eQTL/data/example/sample_output.txt', sep='\t')

In [16]:
x.head()

,gene_id,variant_id,pvals,slope,tvals
0,ENSG00000065325.8,X_1287943_C_G,0.112704,0.012538,1.590184
1,ENSG00000065325.8,19_7674782_C_G,0.018718,-0.016359,-2.362213
2,ENSG00000065325.8,11_8682249_C_G,0.829974,0.001619,0.214897
3,ENSG00000069011.11,20_12222898_C_G,0.054022,-0.268060,-1.933223
4,ENSG00000069011.11,14_12485031_C_G,0.099921,-0.234032,-1.649643
